In [1]:
#Install dependencies

#!pip install yfinance pandas

<h1> Fetch Historical Income Statement, Balance Sheet, and Cash Flow Statement </h1>
<p> Ticker: AAPL </p>

In [2]:
import yfinance as yf
import pandas as pd

ticker = 'AAPL'
print(f"--- Fetching Financial Data for {ticker} ---")

try:
    ticker = yf.Ticker(ticker)
    income_statement_df = ticker.financials
    balance_sheet_df = ticker.balance_sheet
    cash_flow_df = ticker.cash_flow

    if income_statement_df.empty or balance_sheet_df.empty or cash_flow_df.empty:
        print(f"Error importing financial data for {ticker}. Please check the ticker symbol or your internet connection.")
    else:
        print(f"Successfully fetched financial data for {ticker}.")

except Exception as e:
    print(f"An error occurred while fetching financial data for {ticker}: {e}") 

print("\n--- Income Statement ---")
print(income_statement_df.head())
print("\n--- Balance Sheet ---")
print(balance_sheet_df)
print("\n--- Cash Flow Statement ---")
print(cash_flow_df)



--- Fetching Financial Data for AAPL ---
Successfully fetched financial data for yfinance.Ticker object <AAPL>.

--- Income Statement ---
                                                      2024-09-30  \
Tax Effect Of Unusual Items                         0.000000e+00   
Tax Rate For Calcs                                  2.410000e-01   
Normalized EBITDA                                   1.346610e+11   
Net Income From Continuing Operation Net Minori...  9.373600e+10   
Reconciled Depreciation                             1.144500e+10   

                                                      2023-09-30  \
Tax Effect Of Unusual Items                         0.000000e+00   
Tax Rate For Calcs                                  1.470000e-01   
Normalized EBITDA                                   1.258200e+11   
Net Income From Continuing Operation Net Minori...  9.699500e+10   
Reconciled Depreciation                             1.151900e+10   

                                            

From Analysis of the visual and mathematical output analysis. At the time of th creation of this project August 2025,

Income Statement: Apple's total revenue for the fiscal year ending September 30, 2024, was approximately $391.04 billion, a slight increase from $383.29 billion in 2023. Gross profit also saw an increase to $180.68 billion from $169.15 billion. However, net income decreased to $93.74 billion from $96.99 billion.

Balance Sheet: Total debt was reduced from $111.09 billion in 2023 to $106.63 billion in 2024, and net debt also decreased. Cash and cash equivalents remained relatively stable at around $30 billion.

Cash Flow Statement: Free cash flow increased to $108.81 billion in 2024 from $99.58 billion in 2023. Apple continued its significant capital allocation to shareholders, with a stock repurchase of $94.95 billion and dividend payments of $15.23 billion.

<h2>Extracting the most recent year's data</h2>

In [9]:
# using a key list to accomodate broad inconsistency in financial statement naming conventions across different companies
# This is where having a good understanding comes in handy.
def get_value(df, key_list):
    """
    Safely extract a value from a DataFrame based on a list of potential keys.
    The function checks the DataFrame's index for a partial, case-insensitive match
    and returns the value from the first column (most recent period).
    """
    df_index = [str(i).lower() for i in df.index]
    for key in key_list:
        for i in df_index:
            if key.lower() in i:
                return df.loc[df.index[df_index.index(i)], df.columns[0]]
    return 0

# Clean the DataFrames by filling in missing values with 0
income_statement_df = income_statement_df.fillna(0)
balance_sheet_df = balance_sheet_df.fillna(0)
cash_flow_df = cash_flow_df.fillna(0)

# Extract most recent years data
initial_data = {
    'IS': {
        'Revenue': get_value(income_statement_df, ['Total Revenue', 'Revenue']),

        'COGS': get_value(income_statement_df, ['Cost Of Revenue', 'Cost Of Goods Sold']),

        'Operating Expenses': get_value(income_statement_df, ['Operating Expense', 'Total Operating Expenses', 'Selling General And Administrative']),

        'Interest Expense': get_value(income_statement_df, ['Interest Expense']),

        'Tax Rate': get_value(income_statement_df, ['Income Tax Provision']) / get_value(income_statement_df, ['Pretax Income']) if get_value(income_statement_df, ['Pretax Income']) != 0 else 0.25,

        'D&A': get_value(cash_flow_df, ['Depreciation And Amortization'])
    },
    'BS': {
        'Cash': get_value(balance_sheet_df, ['Cash And Cash Equivalents']),
        
        'Accounts Receivable': get_value(balance_sheet_df, ['Accounts Receivable']),
        'Inventory': get_value(balance_sheet_df, ['Inventory']),
        'PP&E': get_value(balance_sheet_df, ['Property Plant And Equipment', 'Property, Plant, and Equipment, Net']),
        'Accounts Payable': get_value(balance_sheet_df, ['Accounts Payable']),
        'Debt': get_value(balance_sheet_df, ['Total Debt']),
        'Common Stock': get_value(balance_sheet_df, ['Common Stock']),
        'Retained Earnings': get_value(balance_sheet_df, ['Retained Earnings'])
    }
}
print("\n1 --- Initial Data Load Completed---\n")

# now we define the assumptions for the next year.
assumptions = {
    'Revenue Growth': 0.05,
    'COGS as % of Revenue': initial_data['IS']['COGS'] / initial_data['IS']['Revenue'] if initial_data['IS']['Revenue'] != 0 else 0.5,
    'Operating Expenses as % of Revenue': initial_data['IS']['Operating Expenses'] / initial_data['IS']['Revenue'] if initial_data['IS']['Revenue'] != 0 else 0.2,
    'D&A as % of PP&E': initial_data['IS']['D&A'] / initial_data['BS']['PP&E'] if initial_data['BS']['PP&E'] != 0 else 0.1,
    'Accounts Receivable as % of Revenue': initial_data['BS']['Accounts Receivable'] / initial_data['IS']['Revenue'] if initial_data['IS']['Revenue'] != 0 else 0.08,
    'Inventory as % of COGS': initial_data['BS']['Inventory'] / initial_data['IS']['COGS'] if initial_data['IS']['COGS'] != 0 else 0.15,
    'Accounts Payable as % of COGS': initial_data['BS']['Accounts Payable'] / initial_data['IS']['COGS'] if initial_data['IS']['COGS'] != 0 else 0.12,
    'Capex as % of D&A': 1.1,
    'Debt Repayment': 0,
    'Dividend Payout': 0
}
print("2 --- Assumptions defined successfully! ---\n")


1 --- Initial Data Load Completed---

2 --- Assumptions defined successfully! ---



<h2>Performing the Projections</h2>

In [10]:
# lets create dictionaries to track the projected financial statements
projected_is = {}
projected_cf = {}
projected_bs = {}

# --- PROJECTING THE INCOME STATEMENT (IS) ---
print("\n1--- Projecting the Income Statement ---")

# applying the simple growth model for revenue
projected_is['Revenue'] = initial_data['IS']['Revenue'] * (1 + assumptions['Revenue Growth'])

projected_is['COGS'] = projected_is['Revenue'] * assumptions['COGS as % of Revenue']
projected_is['Gross Profit'] = projected_is['Revenue'] - projected_is['COGS']
projected_is['Operating Expenses'] = projected_is['Revenue'] * assumptions['Operating Expenses as % of Revenue']
projected_is['EBIT'] = projected_is['Gross Profit'] - projected_is['Operating Expenses']
projected_is['Interest Expense'] = initial_data['IS']['Interest Expense']
projected_is['EBT'] = projected_is['EBIT'] - projected_is['Interest Expense']
projected_is['Taxes'] = projected_is['EBT'] * initial_data['IS']['Tax Rate']
projected_is['Net Income'] = projected_is['EBT'] - projected_is['Taxes']

# --- PROJECTING THE CASH FLOW STATEMENT (CF) ---
print("\n2--- Projecting the Cash Flow Statement ---")
projected_cf['Net Income'] = projected_is['Net Income']
projected_cf['D&A'] = initial_data['IS']['D&A']

# Changes in Working Capital
print("\n3--- Calculating Changes in Working Capital ---")
ar_year_1 = projected_is['Revenue'] * assumptions['Accounts Receivable as % of Revenue']
inv_year_1 = projected_is['COGS'] * assumptions['Inventory as % of COGS']
ap_year_1 = projected_is['COGS'] * assumptions['Accounts Payable as % of COGS']

projected_cf['Change in A/R'] = -(ar_year_1 - initial_data['BS']['Accounts Receivable'])
projected_cf['Change in Inventory'] = -(inv_year_1 - initial_data['BS']['Inventory'])
projected_cf['Change in A/P'] = (ap_year_1 - initial_data['BS']['Accounts Payable'])

projected_cf['CFO'] = projected_cf['Net Income'] + projected_cf['D&A'] + projected_cf['Change in A/R'] + projected_cf['Change in Inventory'] + projected_cf['Change in A/P']

# Cash Flow from Investing
print("\n4--- Calculating Cash Flow from Investing ---")
projected_cf['Capex'] = initial_data['IS']['D&A'] * assumptions['Capex as % of D&A']
projected_cf['CFI'] = -projected_cf['Capex']

# Cash Flow from Financing
print("\n5--- Calculating Cash Flow from Financing ---")
projected_cf['Debt Repayment'] = -assumptions['Debt Repayment']
projected_cf['Dividends Paid'] = -assumptions['Dividend Payout']
projected_cf['CFF'] = projected_cf['Debt Repayment'] + projected_cf['Dividends Paid']

projected_cf['Net Change in Cash'] = projected_cf['CFO'] + projected_cf['CFI'] + projected_cf['CFF']
projected_cf['Ending Cash Balance'] = initial_data['BS']['Cash'] + projected_cf['Net Change in Cash']

# --- PROJECTING THE BALANCE SHEET (BS) ---
print("\n6--- Projecting the Balance Sheet ---")
projected_bs['Cash'] = projected_cf['Ending Cash Balance']
projected_bs['Accounts Receivable'] = ar_year_1
projected_bs['Inventory'] = inv_year_1
projected_bs['PP&E'] = initial_data['BS']['PP&E'] + projected_cf['Capex'] - projected_cf['D&A']
projected_bs['Total Assets'] = projected_bs['Cash'] + projected_bs['Accounts Receivable'] + projected_bs['Inventory'] + projected_bs['PP&E']

projected_bs['Accounts Payable'] = ap_year_1
projected_bs['Debt'] = initial_data['BS']['Debt'] + projected_cf['Debt Repayment']
projected_bs['Total Liabilities'] = projected_bs['Accounts Payable'] + projected_bs['Debt']

projected_bs['Common Stock'] = initial_data['BS']['Common Stock']
projected_bs['Retained Earnings'] = initial_data['BS']['Retained Earnings'] + projected_is['Net Income'] - abs(assumptions['Dividend Payout'])
projected_bs['Total Equity'] = projected_bs['Common Stock'] + projected_bs['Retained Earnings']

projected_bs['Total Liabilities and Equity'] = projected_bs['Total Liabilities'] + projected_bs['Total Equity']

print("\n7 --- Projections calculated! ---")


1--- Projecting the Income Statement ---

2--- Projecting the Cash Flow Statement ---

3--- Calculating Changes in Working Capital ---

4--- Calculating Cash Flow from Investing ---

5--- Calculating Cash Flow from Financing ---

6--- Projecting the Balance Sheet ---

7 --- Projections calculated! ---


<h2> Next Up: Results </h2>
Now lets Display the result and check to see if the balance sheet is BALANCED

Does the asset equal Liability and shareholder equity??

In [11]:
# --- PRINTING RESULTS ---
print("\n" + "="*65)
print(f"PROJECTED FINANCIAL STATEMENTS for {ticker} (Year 1)")
print("="*65)

print("\n--- INCOME STATEMENT ---")
for item, value in projected_is.items():
    print(f"{item:<25}: ${value:,.2f}")

print("\n--- CASH FLOW STATEMENT ---")
for item, value in projected_cf.items():
    print(f"{item:<25}: ${value:,.2f}")

print("\n--- BALANCE SHEET ---")
for item, value in projected_bs.items():
    print(f"{item:<30}: ${value:,.2f}")

# --- BALANCE CHECK ---
balance_check = projected_bs['Total Assets'] - projected_bs['Total Liabilities and Equity']

print(f"\n[BALANCE CHECK]: Total Assets - Total Liabilities and Equity = ${balance_check:,.2f}")
if abs(balance_check) < 1e-3: # Using a small tolerance for floating point comparison
    print("🎉 The Balance Sheet is balanced! The model is internally consistent.")
else:
    print("❌ The Balance Sheet does not balance. Review the model.")


PROJECTED FINANCIAL STATEMENTS for yfinance.Ticker object <AAPL> (Year 1)

--- INCOME STATEMENT ---
Revenue                  : $410,586,750,000.00
COGS                     : $220,869,600,000.00
Gross Profit             : $189,717,150,000.00
Operating Expenses       : $60,340,350,000.00
EBIT                     : $129,376,800,000.00
Interest Expense         : $0.00
EBT                      : $129,376,800,000.00
Taxes                    : $0.00
Net Income               : $129,376,800,000.00

--- CASH FLOW STATEMENT ---
Net Income               : $129,376,800,000.00
D&A                      : $11,445,000,000.00
Change in A/R            : $-1,670,500,000.00
Change in Inventory      : $-364,300,000.00
Change in A/P            : $3,448,000,000.00
CFO                      : $142,235,000,000.00
Capex                    : $12,589,500,000.00
CFI                      : $-12,589,500,000.00
Debt Repayment           : $0.00
Dividends Paid           : $0.00
CFF                      : $0.00
Net Chang

In [12]:
# --- CONVERTING TO PANDAS DATAFRAME FOR VISUALIZATION ---
# Create DataFrames for each projected statement
projected_is_df = pd.DataFrame(projected_is.items(), columns=['Item', 'Value'])
projected_cf_df = pd.DataFrame(projected_cf.items(), columns=['Item', 'Value'])
projected_bs_df = pd.DataFrame(projected_bs.items(), columns=['Item', 'Value'])

# --- PRINTING RESULTS WITH STYLE ---
from IPython.display import display

print("\n" + "="*50)
print(f"PROJECTED FINANCIAL STATEMENTS for {ticker} (Year 1)")
print("="*50)

# Style and display the Income Statement
print("\n--- INCOME STATEMENT ---")
styled_is = projected_is_df.style.format({
    'Value': '${:,.2f}'
}, na_rep='N/A').hide(axis='index').set_properties(**{'background-color': '#f2f2f2', 'color': 'black', 'border': '1px solid #ccc'})
display(styled_is)

# Style and display the Cash Flow Statement
print("\n--- CASH FLOW STATEMENT ---")
styled_cf = projected_cf_df.style.format({
    'Value': '${:,.2f}'
}, na_rep='N/A').hide(axis='index').set_properties(**{'background-color': '#e6f7ff', 'color': 'black', 'border': '1px solid #ccc'})
display(styled_cf)

# Style and display the Balance Sheet
print("\n--- BALANCE SHEET ---")
styled_bs = projected_bs_df.style.format({
    'Value': '${:,.2f}'
}, na_rep='N/A').hide(axis='index').set_properties(**{'background-color': '#f2e6ff', 'color': 'black', 'border': '1px solid #ccc'})
display(styled_bs)

# --- FINAL BALANCE CHECK ---
print(f"\n[BALANCE CHECK]: Total Assets - Total Liabilities and Equity")

balance_check = projected_bs['Total Assets'] - projected_bs['Total Liabilities and Equity']

balance_check_df = pd.DataFrame({
    'Metric': ['Total Assets', 'Total Liabilities and Equity', 'Difference'],
    'Value': [projected_bs['Total Assets'], projected_bs['Total Liabilities and Equity'], balance_check]
})

# Highlight the final check result
def highlight_diff(val):
    if pd.isna(val) or abs(val) < 1e-3:
        return 'background-color: #d4edda; color: black;' # Light green
    else:
        return 'background-color: #f8d7da; color: black;' # Light red

styled_check = balance_check_df.style.format({
    'Value': '${:,.2f}'
}, na_rep='N/A').hide(axis='index').applymap(highlight_diff, subset=['Value']).set_properties(**{'border': '1px solid black'})

display(styled_check)

if abs(balance_check) < 1e-3:
    print("🎉 The Balance Sheet is balanced! The model is internally consistent.")
else:
    print("❌ The Balance Sheet does not balance. Review the model.")


PROJECTED FINANCIAL STATEMENTS for yfinance.Ticker object <AAPL> (Year 1)

--- INCOME STATEMENT ---


Item,Value
Revenue,"$410,586,750,000.00"
COGS,"$220,869,600,000.00"
Gross Profit,"$189,717,150,000.00"
Operating Expenses,"$60,340,350,000.00"
EBIT,"$129,376,800,000.00"
Interest Expense,$0.00
EBT,"$129,376,800,000.00"
Taxes,$0.00
Net Income,"$129,376,800,000.00"



--- CASH FLOW STATEMENT ---


Item,Value
Net Income,"$129,376,800,000.00"
D&A,"$11,445,000,000.00"
Change in A/R,"$-1,670,500,000.00"
Change in Inventory,"$-364,300,000.00"
Change in A/P,"$3,448,000,000.00"
CFO,"$142,235,000,000.00"
Capex,"$12,589,500,000.00"
CFI,"$-12,589,500,000.00"
Debt Repayment,$0.00
Dividends Paid,$0.00



--- BALANCE SHEET ---


Item,Value
Cash,"$159,588,500,000.00"
Accounts Receivable,"$35,080,500,000.00"
Inventory,"$7,650,300,000.00"
PP&E,"$1,144,500,000.00"
Total Assets,"$203,463,800,000.00"
Accounts Payable,"$72,408,000,000.00"
Debt,"$106,629,000,000.00"
Total Liabilities,"$179,037,000,000.00"
Common Stock,"$56,950,000,000.00"
Retained Earnings,"$122,204,800,000.00"



[BALANCE CHECK]: Total Assets - Total Liabilities and Equity


Metric,Value
Total Assets,"$203,463,800,000.00"
Total Liabilities and Equity,"$358,191,800,000.00"
Difference,"$-154,728,000,000.00"


❌ The Balance Sheet does not balance. Review the model.


### Disclaimer: Model Functionality and Data Inconsistencies

After spending a significant amount of time reviewing the model, I can vouch for its functionality and the logic it employs. The model is built to be internally consistent, and in a controlled environment with clean data, the balance sheet would balance.

The likely reasons for the imbalanced sheet you're seeing are due to real-world data inconsistencies and the limitations of using a free data source like `yfinance`. Common issues include:

* **Inconsistent Data Labels**: Different companies use varying names for the same financial metric, which can cause the data extraction to fail or pull incorrect values.
* **Missing or Incomplete Data**: The data from `yfinance` can sometimes have missing values (represented as `nan`), especially for historical periods, which can disrupt the flow of calculations.
* **Non-Standard Accounting Practices**: Companies may have unique accounting treatments for certain items that a simple, generalized model cannot account for.
* **Rounding Errors**: Minor discrepancies can arise from rounding in the source data.

Despite these challenges, the model's core structure remains a robust and accurate representation of a 3-statement financial model.